# A Hybrid WiFi/Bluetooth RSS Dataset with LS-based localization algorithm

In [250]:
# Imports
import pandas as pd
import numpy as np
import pprint
import math

In [251]:
def read_data(anchors, directory_path):
    """
    Reads the csv-files for one specified measurement case and adds them into a dict of dataframes.
    
    Parameters:
    anchors (list): List of anchors, for example ["Anchor 1", "Anchor 2", "Anchor 3"...]
    directory path (string): The relative directory path of the folder which contains the wanted anchor-csv.files.

    returns: 
    scenario_dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.
    """
    scenario_dataframes = {}

    for anchor in anchors:
        current_df = pd.read_csv(directory_path + anchor + ".csv", sep=';')
        scenario_dataframes[anchor.replace(".csv","")] = current_df
    
    return scenario_dataframes


In [252]:
def calculate_smallest_dataset(dataframes):
    """
    Calculates the minimum over all dataframes' length.

    Parameters:
    dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.

    Returns:
    minimum_length (int): an integer which value represents the minimum over all dataframes' length.
    """

    sizes = []

    for anchor in dataframes:
        df = dataframes[anchor]
        sizes.append(len(df))

    return min(sizes)
        

In [253]:
def calculate_burst_quantity(length, burst_size):
    """
    Calculates the amount of bursts over all dataframes.

    Parameters:
    burst_size (int): An integer which value represents the size of a burst, e.g. the number of consecutive samples.
    length (int): An integer which value represents the number of data to read in the dataset.

    Returns:
    bursts (int): An integer which value represents the amount of bursts / configurations.
    """

    bursts = length // burst_size
    end = burst_size * bursts

    if (end < length):
        bursts = bursts + 1
    
    return bursts

In [254]:
def calculate_target_anchors_distance(dataframes):
    """
    Calculates the distance between the target and the anchors using the Euclidean norm.
    This is the implementation of (1) on the reference paper.

    Parameters:
    dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.

    Returns:
    target_anchors_distance (list): Returns a list of distances between the target and the anchor.
    """

    target_anchors_distance = []

    for anchor in dataframes:
        df = dataframes[anchor]
        anchor_coords = [eval(i) for i in df["Relative Coordinates [m]"][0].split(", ")]
        target_coords = [eval(i) for i in df["Target Coordinates [m]"][0].split(", ")]
        target_anchors_distance.append(math.dist(anchor_coords, target_coords))

    # print(target_anchors_distance)
    return target_anchors_distance

# LINKS
# https://www.w3schools.com/python/ref_math_dist.asp
# https://www.w3schools.com/python/ref_string_split.asp
# https://www.geeksforgeeks.org/python-converting-all-strings-in-list-to-integers/


In [255]:
def calculate_mean_values(scenario_dataframe, length, burst_size): 
    """
    Calculates the mean value over a given amount of bursts.
    This is the implementation of (3) on the reference paper.

    Parameters:
    scenario_dataframe (DataFrame): A DataFrame-object containing the dataset of one anchor.
    length (int): An integer which value represents the number of data to read in the dataset.
    burst_size (int): An integer which value represents the size of a burst, e.g. the number of consecutive samples.

    Returns:
    mean_values (list): Returns a list of mean values over the given amount of bursts.
    """
    
    mean_values = []
    power_values = scenario_dataframe["Rx Power [dBm]"].to_list()
    bursts = calculate_burst_quantity(length, burst_size)

    start = 0
    for i in range(bursts):
        end = start + burst_size
        # print("start: ", start)
        if (end <= length):
            # print("end: ", end)
            mean_values.append(round(np.mean(power_values[start:end]), 3))
        else:
            # print("end: ", length)
            mean_values.append(round(np.mean(power_values[start:length]), 3))
        start = end

    return mean_values

In [256]:
def calculate_target_anchors_estimation(reference_power, path_loss_exp, mean_values):
    """
    Calculates the distance estimates between each anchor and the target.
    This is the implementation of (4) on the reference paper.

    Parameters:
    reference_power (int): The initial RSS value for the distance estimation equation.
    path_loss_exp (int): The "path loss exponent" for the distance estimation formula.
    mean_values (list): A list which values are mean values over the given amount of bursts.

    Returns:
    estimated_distances (list): Returns a list which values are the distance estimations.
    """
    
    estimated_distances = []
    index = 0
    
    for mean in mean_values:
        num = mean - reference_power
        den = -10 * path_loss_exp
        estimated_distances.append(round(10**(num/den), 3))
        index = index + 1

    return estimated_distances


In [257]:
def calculate_A_matrix():
    pass

In [258]:
def calculate_b_vector(n_anchors, configuration):
    """
    Calculates the 'b' vector of the system of equations which the WLS method solves.
    This is the implementation of (5) on the reference paper.

    Parameters:
    n_anchors (int): An integer which value represents the number of anchors in dataset.
    configuration (dict): A dict which keys are the anchor names and values are the respective configurations.

    Returns:
    b (numpy.array): Returns a 1D numpy.array which values are elements of the 'b' vector for the selected configuration.
    """
    
    # A list containing one real number for each anchor.
    # That value is the distance estimation between each anchor and the target.
    local_estimated_distances = configuration["Estimated Distance"]
    
    # A list containing one list for each anchor.
    # Those lists are the 2D-coordinates of each anchor.
    local_anchor_coords = configuration["Coordinates"]
    norm_coords = []

    b = []

    for i in range(n_anchors):
        # print(local_estimated_distances[i])
        sqr_distance = math.pow(local_estimated_distances[i], 2)
        norm_coords.append(np.linalg.norm(local_anchor_coords))
        sqr_coords = math.pow(norm_coords[i], 2)
        diff = sqr_distance - sqr_coords
        b.append(round(diff, 3))
        i = i + 1
    
    return np.array(b)

# https://www.digitalocean.com/community/tutorials/norm-of-vector-python
# https://numpy.org/doc/stable/reference/generated/numpy.append.html

In [259]:
def calculate_W_matrix(n_anchors, path_loss_exp, configuration):
    """
    Calculates, for the selected configuration, the 'W' diagonal matrix.
    The elements are the inverse of the variance of the square of the estimated distance.
    This is the implementation of [11, eq. (14)].

    Parameters:
    n_anchors (int): An integer which value represents the number of anchors in dataset.
    configuration (dict): A dict which keys are the anchor names and values are the respective configurations.

    Returns:
    W (numpy.array): Returns a 2D numpy.array which values are elements of the 'W' diagonal matrix for the selected configuration.
    """

    # A list containing one real number for each anchor.
    # That value is the distance estimation between each anchor and the target.
    local_estimated_distances = configuration["Estimated Distance"]

    standard_deviation = 2
    num = math.pow(standard_deviation, 2)
    den = 4.715 * math.pow(path_loss_exp, 2)
    exp_value = math.exp(num/den)
    var_inverse = []

    for i in range(n_anchors):
        e4_distance = math.pow(local_estimated_distances[i], 4)
        var = e4_distance * exp_value * (exp_value-1)
        var_inverse.append(round(math.pow(var, -1), 3))

    return np.diag(var_inverse)
    
# https://www.w3schools.com/python/ref_math_exp.asp
# https://numpy.org/doc/stable/reference/generated/numpy.diag.html

In [260]:
def calculate_WLS_output():
    pass

# https://numpy.org/doc/stable/reference/generated/numpy.matmul.html

In [261]:
directory_path = "HybridDataset_for_RSSbasedLocalization-main/Scenario A/RPI/RSS_BLT_Dataset/"
anchors = ["Anchor 1", "Anchor 2", "Anchor 3", "Anchor 4", "Anchor 5", "Anchor 6"]
dataframes = read_data(anchors, directory_path)
length = calculate_smallest_dataset(dataframes)

burst_size = 10
burst_quantity = calculate_burst_quantity(length, burst_size)
n_anchors = len(anchors)
reference_power = -40
path_loss_exp = 4

# print("length: ", length, ", burst_quantity: ", burst_quantity, ", burst_size: ", burst_size)
    
# print("--- ", anchor, " ---")
# print("")
# print("Mean values: ", mean_values[anchor])
# print("")
# print("Estimated distances:", estimated_distances[anchor])
# print("")
# print("-----")
# print("")

calculate_mean_values(dataframes["Anchor 1"], length, burst_size)

configurations = {}

for index in range(burst_quantity):
    configurations[index] = {}
    configurations[index]["Mean Power"] = []
    configurations[index]["Estimated Distance"] = []
    configurations[index]["Actual Distance"] = []
    configurations[index]["Coordinates"] = []
    for anchor in dataframes:
        mean_values[anchor] = calculate_mean_values(dataframes[anchor], length, burst_size)
        configurations[index]["Mean Power"].append(mean_values[anchor][index])
        configurations[index]["Estimated Distance"].append(calculate_target_anchors_estimation(reference_power, path_loss_exp, mean_values[anchor]))
        configurations[index]["Actual Distance"].append(dataframes[anchor]["Distance Target - Anchor [m]"][0])
        configurations[index]["Coordinates"].append([eval(i) for i in dataframes[anchor]["Relative Coordinates [m]"][0].split(", ")])

for index in range(burst_quantity):
    pass
    # configurations[index]["B vector"] = []
    # configurations[index]["W matrix"] = []
    # configurations[index]["B vector"].append(calculate_b_vector(n_anchors, configurations[index]))
    # configurations[index]["W matrix"].append(calculate_W_matrix(n_anchors, path_loss_exp, configurations[index]))

pprint.pprint(configurations)

{0: {'Actual Distance': [15.994, 15.994, 10.55, 15, 12.944, 12.944],
     'Coordinates': [[5.55, 15],
                     [5.55, 15],
                     [-10.55, 0],
                     [0, -15],
                     [-10.55, -7.5],
                     [-10.55, -7.5]],
     'Estimated Distance': [[6.237,
                             6.722,
                             6.494,
                             6.026,
                             5.991,
                             6.273,
                             6.8,
                             6.273,
                             5.721,
                             6.06,
                             5.957,
                             5.591,
                             6.026,
                             5.991,
                             6.645,
                             6.494,
                             6.095,
                             6.095,
                             6.026,
                             6.202,
        